In [1]:
import numpy as np
from mpmath import mp
from tqdm import tqdm
import sys, os
# sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'lib')))
# from renormalization_hp import *
mp.dps = 100  # Set desired precision

plus_configs = [
    ([1,1,1], lambda J1, J2: 2*J1 + J2, mp.mpf('3')),
    ([1,1,-1], lambda J1, J2: -J2, mp.mpf('1')),
    ([1,-1,1], lambda J1, J2: -2*J1 + J2, mp.mpf('1')),
    ([-1,1,1], lambda J1, J2: -J2, mp.mpf('1'))
]

minus_configs = [
    ([-1,-1,-1], lambda J1, J2: 2*J1 + J2, mp.mpf('-3')),
    ([-1,-1,1], lambda J1, J2: -J2, mp.mpf('-1')),
    ([-1,1,-1], lambda J1, J2: -2*J1 + J2, mp.mpf('-1')),
    ([1,-1,-1], lambda J1, J2: -J2, mp.mpf('-1'))
]

def get_J(d, J0, n):
    if d <= 0:
        return mp.mpf(0)
    return mp.mpf(J0) / mp.power(mp.mpf(d), mp.mpf(n))

def mp_logsumexp(lst):
    if not lst:
        return mp.ninf
    a_max = max(lst)
    if mp.isinf(a_max):
        return a_max
    tmp = sum(mp.exp(a - a_max) for a in lst)
    return a_max + mp.log(tmp)

def compute_R_logs(start, J_func, h):
    J1 = J_func(1)
    J2 = J_func(2)
    distances = [(iL, iR, (start + iR) - (1 + iL)) for iL in range(3) for iR in range(3)]

    def get_totals(left_confs, right_confs):
        totals = []
        for spinsL, El_func, sumL in left_confs:
            El = El_func(J1, J2)
            for spinsR, Er_func, sumR in right_confs:
                Er = Er_func(J1, J2)
                Eint = mp.mpf('0')
                for iL, iR, d in distances:
                    Eint += mp.mpf(spinsL[iL] * spinsR[iR]) * J_func(d)
                total = El + Er + Eint + h * (sumL + sumR)
                totals.append(total)
        return totals

    totals_pp = get_totals(plus_configs, plus_configs)
    totals_pm = get_totals(plus_configs, minus_configs)
    totals_mp = get_totals(minus_configs, plus_configs)
    totals_mm = get_totals(minus_configs, minus_configs)

    log_R_pp = mp_logsumexp(totals_pp)
    log_R_pm = mp_logsumexp(totals_pm)
    log_R_mp = mp_logsumexp(totals_mp)
    log_R_mm = mp_logsumexp(totals_mm)

    return log_R_pp, log_R_pm, log_R_mp, log_R_mm

def compute_J_prime_func(start, J_func, h):
    log_pp, log_pm, log_mp, log_mm = compute_R_logs(start, J_func, h)
    if mp.isinf(log_pm) or mp.isinf(log_mp):
        return mp.inf
    ln_arg = log_pp + log_mm - log_pm - log_mp
    return mp.mpf('0.25') * ln_arg

def compute_H_prime_func(J_func, h):
    start = 4  # For r'=1
    log_pp, _, _, log_mm = compute_R_logs(start, J_func, h)
    ln_arg = log_pp - log_mm
    return mp.mpf('0.25') * ln_arg

def generate_rg_flow_no_viz(J0, n, max_k, num_steps):
    J_func = lambda d: get_J(d, J0, n) if d > 0 else mp.mpf(0)
    rs = list(range(1, max_k + 1))
    initial_Js = [J_func(r) for r in rs]
    all_Js = [initial_Js]
    h = mp.mpf(0)  # Fixed to 0 for the flow
    for step in range(1, num_steps + 1):
        Jps = []
        for r in rs:
            start = 3 * r + 1
            Jp = compute_J_prime_func(start, J_func, h)
            Jps.append(Jp)
        all_Js.append(Jps)
        J_dict = {r: Jps[r-1] for r in rs}
        J_func = lambda d: J_dict.get(d, mp.mpf(0)) if d > 0 else mp.mpf(0)
    return all_Js

def construct_recursion_matrix(J0, n, max_k, num_steps, matrix_size=5, epsilon=1e-6):
    all_Js = generate_rg_flow_no_viz(J0, n, max_k, num_steps)
    if len(all_Js) < 2:
        raise ValueError("Need at least 2 steps for derivatives")

    # Extract first matrix_size from last two steps
    J_k_minus_1 = all_Js[-2][:matrix_size]  # list of mp.mpf, truncated
    J_k = all_Js[-1][:matrix_size]  # list of mp.mpf, truncated

    M = matrix_size  # Size of matrix
    jac = np.zeros((M, M))

    # Define compute_T truncated to M
    def compute_T(J_vec_mp):
        J_dict = {r+1: J_vec_mp[r] for r in range(M)}
        J_func = lambda d: J_dict.get(d, mp.mpf(0)) if d > 0 else mp.mpf(0)
        J_prime = []
        h = mp.mpf(0)  # h=0
        for r in range(1, M+1):
            start = 3 * r + 1
            Jp = compute_J_prime_func(start, J_func, h)
            J_prime.append(Jp)
        return J_prime

    epsilon_mp = mp.mpf(epsilon)

    J_k_minus_1_float = np.array([float(j) for j in J_k_minus_1])
    J_k_float = np.array([float(j) for j in J_k])

    for s in range(M):
        J_pert_mp = [j for j in J_k_minus_1]  # copy list of mp
        J_pert_mp[s] += epsilon_mp
        T_pert_mp = compute_T(J_pert_mp)
        T_pert_float = np.array([float(j) for j in T_pert_mp])
        jac[:, s] = (T_pert_float - J_k_float) / float(epsilon)

    return jac

def find_J_c(n, max_k=1000, tol=1e-6, J_low=1e-10, J_high=3.0):
    if n <= 0 or n >= 2:
        raise ValueError("n must be between 0 and 2, excluding the edges.")

    start_track = 3
    max_steps = 5
    tol = mp.mpf(tol)
    J_low = mp.mpf(J_low)
    J_high = mp.mpf(J_high)
    rs = list(range(1, max_k + 1))

    def compute_flow(J0):
        J_func = lambda d: get_J(d, J0, n) if d > 0 else mp.mpf(0)
        all_Js = [[J_func(r) for r in rs]]
        h = mp.mpf(0)
        for step in range(1, max_steps + 1):
            Jps = [compute_J_prime_func(3 * r + 1, J_func, h) for r in rs]
            all_Js.append(Jps)
            if step >= start_track:
                J_r2_current = Jps[1]
                J_r2_previous = all_Js[-2][1]
                if J_r2_current > J_r2_previous:
                    return all_Js, True  # Growing (ferromagnetic)
                if J_r2_current < J_r2_previous:
                    return all_Js, False  # Decaying (paramagnetic)
            J_dict = {r: Jps[r-1] for r in rs}
            J_func = lambda d: J_dict.get(d, mp.mpf(0)) if d > 0 else mp.mpf(0)
        # Fallback
        J_r2_initial = all_Js[0][1]
        J_r2_final = all_Js[-1][1]
        return all_Js, J_r2_final > J_r2_initial

    iter_count = 0
    while J_high - J_low > tol and iter_count < 100:
        iter_count += 1
        J_mid = (J_low + J_high) / 2
        _, is_growing = compute_flow(J_mid)
        if is_growing:
            J_high = J_mid  # Growing: search lower half
        else:
            J_low = J_mid  # Decaying: search upper half
    return (J_low + J_high) / 2

In [2]:
n = 1  # Exponent n
max_k = 1000  # Max distance for flow
num_steps = 5  # Number of RG steps
matrix_size = 5  # Number of first J values for matrix
b = 3.0  # Scale factor

# Find critical Jc
Jc = find_J_c(n, max_k=max_k, tol=1e-6, J_low=1e-10, J_high=3.0)
print(f"Critical Jc for n={n}: {float(Jc)}")

# Compute recursion matrix at Jc
M_matrix = construct_recursion_matrix(Jc, n, max_k, num_steps, matrix_size=matrix_size)
print("Recursion Matrix M:")
print(M_matrix)

# Compute eigenvalues
eigenvalues = np.linalg.eigvals(M_matrix)
eigenvalues = sorted(eigenvalues, key=abs, reverse=True)
lambda_t = eigenvalues[0]
yT = np.log(abs(lambda_t)) / np.log(b) if abs(lambda_t) > 0 else 0
nu = 1 / yT if yT != 0 else np.inf
alpha = 2 - 1 / yT if yT != 0 else -np.inf
print(f"Largest eigenvalue λ_t = {lambda_t:.6f}")
print(f"ν = {nu:.6f}, α = {alpha:.6f}")

# Compute yH, eta, delta
all_Js = generate_rg_flow_no_viz(Jc, n, max_k, num_steps)
J_fixed = all_Js[-1]
J_func_fixed = lambda d: J_fixed[d-1] if 1 <= d <= max_k else mp.mpf(0)

h_small = mp.mpf('1e-30')
H_prime = compute_H_prime_func(J_func_fixed, h_small)
dyh = H_prime / h_small
yH = float(mp.log(dyh) / mp.log(mp.mpf('3')))

eta = 1.0 - yH
delta = yT / (1.0 - yH)

print(f"yH = {yH:.6f}")
print(f"η = {eta:.6f}")
print(f"δ = {delta:.6f}")

Critical Jc for n=1: 0.13968837270734324
Recursion Matrix M:
[[ 5.20986952e-01  8.82475463e-01  1.03124960e+00  7.13482425e-01
   3.06883209e-01]
 [-4.07903685e+04 -4.07903802e+04 -4.07904056e+04 -4.07901034e+04
  -4.07897034e+04]
 [-4.52321945e+04 -4.52321945e+04 -4.52321945e+04 -4.52321945e+04
  -4.52321945e+04]
 [-3.36441486e+04 -3.36441486e+04 -3.36441486e+04 -3.36441486e+04
  -3.36441486e+04]
 [-2.68146264e+04 -2.68146264e+04 -2.68146264e+04 -2.68146264e+04
  -2.68146264e+04]]
Largest eigenvalue λ_t = -146480.385853+0.000000j
ν = 0.092362, α = 1.907638
yH = 0.559059
η = 0.440941
δ = 24.554220


In [ ]:
# Use case: Calculate and plot exponents for multiple n values
n_values = np.linspace(0.75, 1.99, 25)  # Example: 6 values between 0.8 and 1.9; adjust for more/less
max_k = 5000
num_steps = 5
matrix_size = 5
b = 3.0
h_small = mp.mpf('1e-30')

# Lists to store exponents
Jcs = []
nus = []
alphas = []
etas = []
deltas = []

for n in n_values:
    print(f"Processing n = {n:.2f}")
    
    # Find critical Jc
    Jc = find_J_c(n, max_k=max_k, tol=1e-6, J_low=1e-10, J_high=3.0)
    Jcs.append(float(Jc))
    print(f"  Critical Jc: {float(Jc):.6f}")
    
    # Compute recursion matrix at Jc
    M_matrix = construct_recursion_matrix(Jc, n, max_k, num_steps, matrix_size=matrix_size)
    
    # Compute eigenvalues for thermal exponents
    eigenvalues = np.linalg.eigvals(M_matrix)
    eigenvalues = sorted(eigenvalues, key=abs, reverse=True)
    lambda_t = eigenvalues[0]
    yT = np.log(abs(lambda_t)) / np.log(b) if abs(lambda_t) > 0 else 0
    nu = 1 / yT if yT != 0 else np.inf
    alpha = 2 - 1 / yT if yT != 0 else -np.inf
    nus.append(nu)
    alphas.append(alpha)
    print(f"  ν = {nu:.6f}, α = {alpha:.6f}")
    
    # Compute magnetic exponents
    all_Js = generate_rg_flow_no_viz(Jc, n, max_k, num_steps)
    J_fixed = all_Js[-1]
    J_func_fixed = lambda d: J_fixed[d-1] if 1 <= d <= max_k else mp.mpf(0)
    
    H_prime = compute_H_prime_func(J_func_fixed, h_small)
    dyh = H_prime / h_small
    yH = float(mp.log(dyh) / mp.log(mp.mpf('3')))
    
    eta = 1.0 - yH
    delta = yT / (1.0 - yH)
    etas.append(eta)
    deltas.append(delta)
    print(f"  η = {eta:.6f}, δ = {delta:.6f}")

# Plot the results
fig, axs = plt.subplots(2, 2, figsize=(12, 10))
axs[0, 0].plot(n_values, nus, marker='o', label='ν')
axs[0, 0].set_xlabel('n (interaction exponent)')
axs[0, 0].set_ylabel('ν')
axs[0, 0].grid(True)

axs[0, 1].plot(n_values, alphas, marker='o', label='α')
axs[0, 1].set_xlabel('n (interaction exponent)')
axs[0, 1].set_ylabel('α')
axs[0, 1].grid(True)

axs[1, 0].plot(n_values, etas, marker='o', label='η')
axs[1, 0].set_xlabel('n (interaction exponent)')
axs[1, 0].set_ylabel('η')
axs[1, 0].grid(True)

axs[1, 1].plot(n_values, deltas, marker='o', label='δ')
axs[1, 1].set_xlabel('n (interaction exponent)')
axs[1, 1].set_ylabel('δ')
axs[1, 1].grid(True)

plt.suptitle('Critical Exponents vs. Interaction Exponent n')
plt.tight_layout()
plt.show()

Processing n = 0.75
